# Модуль Г. Сетевой анализ данных (Вариатив)

## Задание

**Краткое описание задания:** выявление связей и взаимодействия между элементами с использованием базовых методов дискретной математики, в том числе реализация инструментов визуализации. Определение ключевых элементов графа. Предложение рекомендаций на основе анализа данных.

У вас есть данные о нефтяном месторождении, включающие информацию о координатах скважин, их производительности, глубине, типе нефти и др. Ваша задача – реализовать инструмент для аналитика с функционалом построения сети взаимосвязей между скважинами на основе физических и технических характеристик скважин. Затем провести сетевой анализ для определения наиболее важных узлов сети, оптимальных путей для транспортировки нефти, выявления узких мест и оптимизации логистических процессов.

**Результат модуля:** создание модульного инструмента сетевого анализа с указанием основных метрик и их описанием или визуализация графа с использованием специализированного программного обеспечения и его описанием, а также с рекомендациями (в формате docx)

# 1. Разделение данных на узлы и рёбра

Для реализации инструмента сетевого анализа в Gephi, нужно подготовить данные так, чтобы они могли быть загружены в Gephi для дальнейшего анализа. Для этого мы будем строить сеть на основе взаимосвязей между скважинами по их физическим и техническим характеристикам.

Подготовка данных:

Извлечем координаты скважин.
Построим матрицу смежности, где связи между скважинами будут определяться по определенным правилам (например, по расстоянию или схожести технических характеристик).
Экспорт данных для Gephi:

Создадим два файла: один для узлов (скважин) и один для связей между ними.

In [15]:
import pandas as pd
import re
from geopy.distance import great_circle
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Загрузка данных
file_path = 'oil-and-gas-dataframe.xlsx'
oil_data = pd.read_excel(file_path)

# Функция для извлечения координат из строки
def extract_coordinates(location_str):
    match = re.search(r"\(([^)]+)\)", location_str)
    if match:
        lat, lon = map(float, match.group(1).split(","))
        return lat, lon
    else:
        return None, None

# Извлечение координат
oil_data[['Latitude', 'Longitude']] = oil_data['Location'].apply(lambda x: pd.Series(extract_coordinates(x)))

# Очистка координат от лишних пробелов и символов
oil_data['Latitude'] = oil_data['Latitude'].apply(lambda x: str(x).strip())
oil_data['Longitude'] = oil_data['Longitude'].apply(lambda x: str(x).strip())

# Определение порога расстояния в километрах
distance_threshold = 50  # например, 50 км

# Выборка данных для демонстрационных целей
sample_size = 100  # использование выборки из 100 скважин для быстрого вычисления
sampled_oil_data = oil_data.sample(n=sample_size, random_state=42)
sample_coordinates = sampled_oil_data[['Latitude', 'Longitude']].dropna().values

# Модель ближайших соседей для выборки
nbrs_sampled = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(sample_coordinates)
distances_sampled, indices_sampled = nbrs_sampled.kneighbors(sample_coordinates)

# Создание ребер на основе порога расстояния для выборки
edges_sampled = []

for i, (distance_row, index_row) in enumerate(zip(distances_sampled, indices_sampled)):
    for dist, idx in zip(distance_row, index_row):
        if dist <= distance_threshold and i != idx:
            source = sampled_oil_data.iloc[i]['Location']
            target = sampled_oil_data.iloc[idx]['Location']
            edges_sampled.append((source, target, dist))

# Преобразование списка ребер в DataFrame
edges_df_sampled = pd.DataFrame(edges_sampled, columns=['Source', 'Target', 'Distance'])

# Удаление дублирующихся ребер
edges_df_sampled = edges_df_sampled.drop_duplicates(subset=['Source', 'Target'])

# Подготовка данных для узлов
nodes_df_sampled = sampled_oil_data[['Location', 'Latitude', 'Longitude']].copy()
nodes_df_sampled.rename(columns={'Location': 'Id'}, inplace=True)

# Очистка идентификаторов узлов от лишних пробелов и символов
nodes_df_sampled['Id'] = nodes_df_sampled['Id'].str.strip().str.replace(r'[^a-zA-Z0-9]', '')

# Удаление дублирующихся узлов
nodes_df_unique = nodes_df_sampled.drop_duplicates(subset='Id')

# Сохранение узлов и ребер в CSV файлы
nodes_csv_path = 'nodes_oil_unique.csv'
edges_csv_path = 'edges_oil_unique.csv'

nodes_df_unique.to_csv(nodes_csv_path, index=False, sep=',')
edges_df_sampled.to_csv(edges_csv_path, index=False, sep=',')

print(f'Updated nodes CSV saved to: {nodes_csv_path}')
print(f'Edges CSV saved to: {edges_csv_path}')


Updated nodes CSV saved to: nodes_oil_unique.csv
Edges CSV saved to: edges_oil_unique.csv
